In [1]:
# !pip install groq qdrant-client
# !pip install langchain-groq
# !pip install langchainhub

# Imports

In [2]:
import os
from PyPDF2 import PdfReader
import numpy as np
import groq
from groq import Groq
from qdrant_client import QdrantClient
from qdrant_client.http import models as QM
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from fastembed import TextEmbedding
import numpy as np
import time
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import Tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.llms import OpenAI
from langchain_groq import ChatGroq


# Extracting Text from PDFs

In [3]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for page in reader.pages:
        extracted_text += page.extract_text()
    return extracted_text

def extract_text_from_pdfs_in_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            extracted_text = extract_text_from_pdf(pdf_path)
            txt_filename = os.path.splitext(filename)[0] + ".txt"
            txt_filepath = os.path.join(directory, txt_filename)
            with open(txt_filepath, "w") as txt_file:
                txt_file.write(extracted_text)

# Specify the directory containing PDF files
directory_path = "Docs/"

# Extract text from PDFs in the directory and save as text files
extract_text_from_pdfs_in_directory(directory_path)

# Generating Embedding 

In [4]:
GROQ_API_KEY = "<GROQ_API_KEY>"

collection_name = "legal_contract"

In [5]:
directory_path = "Docs"

# List all .txt files in the directory
txt_files = [file for file in os.listdir(directory_path) if file.endswith('.txt')]

loader = TextLoader(directory_path + "/" + txt_files[0])
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [6]:
# Initialize the TextEmbedding model
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

In [7]:
retriever = qdrant.as_retriever()

# Setting ReAct Agents

In [8]:
# convert PDR retriever into a tool
tool_search = create_retriever_tool(
    retriever,
    "eligibility",
    "Checks for eligibility of the contract",
)

In [9]:
tool_search.func

functools.partial(<function _get_relevant_documents at 0x79656cae45e0>, retriever=VectorStoreRetriever(tags=['Qdrant', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x79656b196290>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')

In [10]:
tool_search.args_schema.schema()

{'title': 'RetrieverInput',
 'description': 'Input to the retriever.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'query to look up in retriever',
   'type': 'string'}},
 'required': ['query']}

In [11]:
def get_age(name: str, person_database: dict) -> int:
    """
    Get the age of a person from the database.

    Args:
    - name (str): The name of the person.
    - person_database (dict): A dictionary containing person information.

    Returns:
    - int: The age of the person if found, otherwise None.
    """
    if name in person_database:
        return person_database[name]["Age"]
    else:
        return None

def get_health(name: str, person_database: dict) -> str:
    """
    Get the health status of a person from the database.

    Args:
    - name (str): The name of the person.
    - person_database (dict): A dictionary containing person information.

    Returns:
    - str: The health status of the person if found, otherwise None.
    """
    if name in person_database:
        return person_database[name]["Health"]
    else:
        return None

def get_age_info(name: str) -> str:
    """
    Get age and health information for a person.

    Args:
    - name (str): The name of the person.

    Returns:
    - str: A string containing age and health information for the person.
    """
    person_database = {
        "Sam": {"Age": 21, "Nationality": "US", "Health": "Good"},
        "Alice": {"Age": 25, "Nationality": "UK", "Health": "Good"},
        "Bob": {"Age": 11, "Nationality": "US", "Health": "Mentally Not Sound"}
    }
    age = get_age(name, person_database)
    if age is not None:
        return f"\nAge: {age}\n"
    else:
        return f"\nAge Information for {name} not found.\n"
    
def get_health_info(name: str) -> str:
    """
    Get age and health information for a person.

    Args:
    - name (str): The name of the person.

    Returns:
    - str: A string containing age and health information for the person.
    """
    person_database = {
        "Sam": {"Age": 21, "Nationality": "US", "Health": "Good"},
        "Alice": {"Age": 25, "Nationality": "UK", "Health": "Good"},
        "Bob": {"Age": 11, "Nationality": "US", "Health": "Mentally Not Sound"}
    }
    health = get_health(name, person_database)
    if health is not None:
        return f"\nHealth: {health}\n"
    else:
        return f"\nHealth Information for {name} not found.\n"

# Define the Tool
get_age_info_tool = Tool(
    name="Get Age",
    func=get_age_info,
    description="Useful for getting age information for any person. Input should be the name of the person."
)
get_health_info_tool = Tool(
    name="Get Health",
    func=get_health_info,
    description="Useful for getting health information for any person. Input should be the name of the person."
)


# Common Agent prompts

In [12]:
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


# Running the ReAct agent

In [14]:
# tools = [tool_search, get_info_tool]

tools = [get_age_info_tool, get_health_info_tool]

model = ChatGroq(model_name="llama3-70b-8192", groq_api_key=GROQ_API_KEY, temperature=0)
# model = OpenAI(openai_api_key="<OPENAI_API>")

react_agent = create_react_agent(model, tools=tools, prompt=prompt_react)
react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

query = "I am Bob. Can i make a legal contract?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: To determine if Bob can make a legal contract, I need to know if Bob is of legal age. In most places, the legal age for entering into contracts is 18 years old.

Action: Get Age
Action Input: Bob
Age: 11
Thought: Since Bob is only 11 years old, he is not of legal age to enter into a contract. In most places, the legal age for entering into contracts is 18 years old.

Final Answer: No, Bob cannot make a legal contract.

> Finished chain.


{'input': 'I am Bob. Can i make a legal contract?',
 'output': 'No, Bob cannot make a legal contract.'}